<a href="https://colab.research.google.com/github/asigalov61/Amazon-Deep-Composer/blob/master/Amazon_Deep_Composer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training a custom AR-CNN model 
In this Jupyter notebook, we guide you through several steps of the data science life cycle. We explain how to acquire the data that you use for this project, 
provide some exploratory data analysis (**EDA**), and show how we augment the data during training. 


### The AWS DeepComposer approach to generating music  
Autoregressive-based approaches are prone to accumulate errors during training. To help mitigate this problem, we train our AR-CNN model so that it can detect and then fix mistakes, including those made by the model itself.

We do this by treating music generation as a series of *edit events*, which can be either the addition or removal of a note. An *edit sequence* is a series of edit events. Every edit sequence can directly correspond to a piano roll.

By training our model to view the problem as edit events rather than as an entire image or just the addition of notes, we found that it can offset the accumulation of errors and generate higher quality music.

Now that you understand the basic theory behind our approach, let’s dive into the code. In the next section, we show examples of the piano roll format that we use for training the model.

## Installing dependencies
First, let's install and import all of the Python packages that we will use in this tutorial.

In [ ]:
# The MIT-Zero License

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.


# Create the environment and install required packages
!git clone https://github.com/asigalov61/aws-deepcomposer-samples.git
%cd /content/aws-deepcomposer-samples/ar-cnn
%tensorflow_version 1.x
!pip install -r requirements.txt
!pip install tensorflow-gpu==1.15.3

In [ ]:
# Imports
import os
import glob
import json
import numpy as np
import keras
from enum import Enum
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, BatchNormalization, Dropout
from keras.optimizers import Adam, RMSprop
from keras import backend as K
from random import randrange
import random
import math
import pypianoroll
from utils.midi_utils import play_midi, plot_pianoroll, get_music_metrics, process_pianoroll, process_midi
from constants import Constants
from augmentation import AddAndRemoveAPercentageOfNotes
from data_generator import PianoRollGenerator
from utils.generate_training_plots import GenerateTrainingPlots
from inference import Inference
from model import OptimizerType
from model import ArCnnModel

## Importing the data 


In [ ]:
#!unzip data/JSB\ Chorales.zip -d data
#@title (Best Choice/Works best stand-alone) Alex Piano Only Drafts Original 1500 MIDIs 
%cd /content/aws-deepcomposer-samples/ar-cnn/input
!wget 'https://github.com/asigalov61/AlexMIDIDataSet/raw/master/AlexMIDIDataSet-CC-BY-NC-SA-All-Drafts-Piano-Only.zip'
!unzip -j 'AlexMIDIDataSet-CC-BY-NC-SA-All-Drafts-Piano-Only.zip'

In [ ]:
#Import the MIDI files from the data_dir and save them with the midi_files variable  
data_dir = '/content/aws-deepcomposer-samples/ar-cnn/input/*.mid'
midi_files = glob.glob(data_dir)

#Finds our random MIDI file from the midi_files variable and then plays it
#Note: To listen to multiple samples from the Bach dataset, you can run this cell over and over again. 
random_midi = randrange(len(midi_files))
play_midi(midi_files[random_midi])

## Preprocessing the data into the *piano roll* format

### Reviewing sample piano rolls 

### Why do we use 128 timesteps?
In this tutorial, we use 8-[bar](https://en.wikipedia.org/wiki/Bar_(music)) samples from the dataset. We subdivide those 8 bars into 128 timesteps. That's because each of the 8 bars contains 4 beats. We further divide each beat into 4 timesteps. 

This yields 128 timesteps:

$$ \frac{4\;timesteps}{1\;beat} * \frac{4\;beats}{1\;bar} * \frac{8\;bars}{1} = 128\;timesteps $$

We found that this level of resolution is sufficient to capture the musical details in our dataset.

### Creating samples of uniform size (shape) for model training 

For model training, the *input piano rolls* must be the same size. As you saw when we used the `play_midi` function, each sample isn't the same length. We use two functions to create *target piano rolls* that are the same size: `process_midi` and `process_pianoroll`. These functions are wrapped in a larger function, `generate_samples`, which also takes in constants that are related to subdividing the .mid files.

#### In the code cells below:
- `generate_samples` is a function used to ingest the midi files and break the files down into a uniform shape
- `plot_pianoroll` uses a built in function `plot_track` from the  [`pypianoroll`](https://salu133445.github.io/pypianoroll/visualization.html) library to plot a piano roll track from the dataset.

In [ ]:
# Generate MIDI file samples
def generate_samples(midi_files, bars, beats_per_bar, beat_resolution, bars_shifted_per_sample):
    """
    dataset_files: All files in the dataset
    return: piano roll samples sized to X bars
    """
    timesteps_per_nbars = bars * beats_per_bar * beat_resolution
    time_steps_shifted_per_sample = bars_shifted_per_sample * beats_per_bar * beat_resolution
    samples = []
    for midi_file in midi_files:
        pianoroll = process_midi(midi_file, beat_resolution) # Parse the MIDI file and get the piano roll
        samples.extend(process_pianoroll(pianoroll, time_steps_shifted_per_sample, timesteps_per_nbars))
    return samples

In [ ]:
# Saving the generated samples into a dataset variable 
dataset_samples = generate_samples(midi_files, Constants.bars, Constants.beats_per_bar,Constants.beat_resolution, Constants.bars_shifted_per_sample)
# Shuffle the dataset
random.shuffle(dataset_samples);

In [ ]:
# Visualize a random piano roll from the dataset 
random_pianoroll = dataset_samples[randrange(len(dataset_samples))]
plot_pianoroll(pianoroll = random_pianoroll,
               beat_resolution = 4)

## Augmenting data during training

### Adding or removing notes during training


### Removing random notes from a target piano roll to create input piano rolls


### Adding random notes to the target piano roll to create input piano rolls 



### Sampling from a uniform distribution


In [ ]:
sampling_lower_bound_remove = 0 
sampling_upper_bound_remove = 100
sampling_lower_bound_add = 1
sampling_upper_bound_add = 1.5

## Calculating the loss function


>**NOTE**: The model can pick any of the notes that were added or removed during data augmentation.

This difference in distributions between the distribution representing the *input piano roll* and the uniform distribution can be calculated as the *Kullback–Leibler divergence*. Our loss function is the difference between the model’s output and the uniform distribution of all of the pixel (note) probabilities in the symmetric difference.

In [ ]:
# Customized loss function
class Loss():
    @staticmethod 
    def built_in_softmax_kl_loss(target, output):
        '''
        Custom Loss Function
        :param target: ground truth values
        :param output: predicted values
        :return kullback_leibler_divergence loss
        '''
        target = K.flatten(target)
        output = K.flatten(output)
        target = target / K.sum(target)
        output = K.softmax(output)
        return keras.losses.kullback_leibler_divergence(target, output)

## Model architecture

### Creating the model architecture

## Training

In [ ]:
dataset_size = len(dataset_samples)
dataset_split = math.floor(dataset_size * Constants.training_validation_split) 

training_samples = dataset_samples[0:dataset_split]
print("training samples length: {}".format(len(training_samples)))
validation_samples = dataset_samples[dataset_split + 1:dataset_size]
print("validation samples length: {}".format(len(validation_samples)))

### Specifying training hyperparameters 

>**NOTE** 
>If you want to test that your model is training on your custom dataset, you can decrease the number of `epochs` down to **1** in the cell below.

In [ ]:
# Piano Roll Input Dimensions
input_dim = (Constants.bars * Constants.beats_per_bar * Constants.beat_resolution, 
             Constants.number_of_pitches, 
             Constants.number_of_channels)
# Number of Filters In The Convolution
num_filters = 32
# Growth Rate Of Number Of Filters At Each Convolution
growth_factor = 2
# Number Of Encoder And Decoder Layers
num_layers = 5
# A List Of Dropout Values At Each Encoder Layer
dropout_rate_encoder = [0, 0.5, 0.5, 0.5, 0.5]
# A List Of Dropout Values At Each Decoder Layer
dropout_rate_decoder = [0.5, 0.5, 0.5, 0.5, 0]
# A List Of Flags To Ensure If batch_normalization Should be performed At Each Encoder
batch_norm_encoder = [True, True, True, True, False]
# A List Of Flags To Ensure If batch_normalization Should be performed At Each Decoder
batch_norm_decoder = [True, True, True, True, False]
# Path to Pretrained Model If You Want To Initialize Weights Of The Network With The Pretrained Model
pre_trained = False
# Learning Rate Of The Model
learning_rate = 0.001
# Optimizer To Use While Training The Model
optimizer_enum = OptimizerType.ADAM
# Batch Size
batch_size = 128
# Number Of Epochs
epochs = 500

In [ ]:
# The Number of Batch Iterations Before A Training Epoch Is Considered Finished
steps_per_epoch = int(
    len(training_samples) * Constants.samples_per_ground_truth_data_item / int(batch_size))

print("The Total Number Of Steps Per Epoch Are: "+ str(steps_per_epoch))

# Total Number Of Time Steps
n_timesteps = Constants.bars * Constants.beat_resolution * Constants.beats_per_bar

### Creating the data generators that perform data augmentation

To create the *input piano rolls* during training, we need data generators for both the training and validation samples. For our purposes, we use a custom data generator to perform data augmentation.

In [ ]:
## Training Data Generator
training_data_generator = PianoRollGenerator(sample_list=training_samples,
                                             sampling_lower_bound_remove = sampling_lower_bound_remove,
                                             sampling_upper_bound_remove = sampling_upper_bound_remove,
                                             sampling_lower_bound_add = sampling_lower_bound_add,
                                             sampling_upper_bound_add = sampling_upper_bound_add,
                                             batch_size = batch_size,
                                             bars = Constants.bars,
                                             samples_per_data_item = Constants.samples_per_ground_truth_data_item,
                                             beat_resolution = Constants.beat_resolution,
                                             beats_per_bar = Constants.beats_per_bar,
                                             number_of_pitches = Constants.number_of_pitches,
                                             number_of_channels = Constants.number_of_channels)

In [ ]:
# Validation Data Generator
validation_data_generator = PianoRollGenerator(sample_list = validation_samples,
                                               sampling_lower_bound_remove = sampling_lower_bound_remove,
                                               sampling_upper_bound_remove = sampling_upper_bound_remove,
                                               sampling_lower_bound_add = sampling_lower_bound_add,
                                               sampling_upper_bound_add = sampling_upper_bound_add,
                                               batch_size = batch_size, 
                                               bars = Constants.bars,
                                               samples_per_data_item = Constants.samples_per_ground_truth_data_item,
                                               beat_resolution = Constants.beat_resolution,
                                               beats_per_bar = Constants.beats_per_bar, 
                                               number_of_pitches = Constants.number_of_pitches,
                                               number_of_channels = Constants.number_of_channels)

### Creating callbacks for the model


In [ ]:
# Callback For Loss Plots 
plot_losses = GenerateTrainingPlots()
## Checkpoint Path
checkpoint_filepath =  'best-model-epoch:{epoch:04d}.hdf5'

# Callback For Saving Model Checkpoints 
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

# Create A List Of Callbacks
callbacks_list = [plot_losses, model_checkpoint_callback]

In [ ]:
# Create A Model Instance
MusicModel = ArCnnModel(input_dim = input_dim,
                        num_filters = num_filters,
                        growth_factor = growth_factor,
                        num_layers = num_layers,
                        dropout_rate_encoder = dropout_rate_encoder,
                        dropout_rate_decoder = dropout_rate_decoder,
                        batch_norm_encoder = batch_norm_encoder,
                        batch_norm_decoder = batch_norm_decoder,
                        pre_trained = pre_trained,
                        learning_rate = learning_rate,
                        optimizer_enum = optimizer_enum)

In [ ]:
model = MusicModel.build_model()

## Starting training 

In the following cell, you start training your model.

>**NOTE**: Training times can vary greatly based on the parameters that you have chosen and the notebook instance type that you chose when launching this notebook. 

In [ ]:
# Start Training
history = model.fit_generator(training_data_generator,
                              validation_data = validation_data_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs = epochs,
                              callbacks = callbacks_list)

## Performing inference 

Congratulations! You have now trained your very own AR-CNN model to generate music. Now you can see how well your model will perform with an input melody. 


### How to change the *inference parameters* when you perform inference 

The model performs inference by sampling from its predicted probability distribution across the entire piano roll. 

Inference is an iterative process. After adding or removing a note from the input, the model feeds this new input back into itself. The model has been trained to both remove and add notes, so it can improve the input melody and correct mistakes that it may have made in earlier iterations.

You also can change the *inference parameters* to observe differences in the quality of the music generated: 

- Sampling iterations (`samplingIterations`): The number of iterations performed during inference. A higher number of sampling iterations gives the model more time to improve the input melody.

- Maximum notes to remove (`maxPercentageOfInitialNotesRemoved`): The maximum percentage of notes that can be removed during inference. Setting this value to 0% prevents the model from removing notes from your input melody.

- Maximum notes to add (`maxNotesAdded`): The maximum percentage of notes that can be added during inference. Setting this value to 0% means no notes will be added to your input melody

>**NOTE:** If you restrict your model's ability to add and remove notes, you risk creating poor compositions. 

- Creativity, `temperature`: To create the output probability distribution, the final layer uses a softmax activation. You can change the temperature for the softmax to produce different levels of creativity in the outputs generated by the model.


#### To change the inference parameters:

1. Open the `inference_parameters.json` file. 
2. Update the variables.
3. Save and close the `inference_parameters.json` file. 
4. Run the following cell.

In [ ]:
# Load The Inference-Related Parameters
with open('inference_parameters.json') as json_file:
    inference_params = json.load(json_file)

### Loading a saved checkpoint file

To use your trained model, you will need to update the `checkpoint_var` variable in the cell below. To see the checkpoint files that you have created, uncomment and then run the following cell.

In [ ]:
!ls -ltr /content/aws-deepcomposer-samples/ar-cnn

In the next code cell, replace the string in variable `checkpoint_var` with the filename, `checkpoints/foo-bar.hdf5` 

In [ ]:
%cd /content/aws-deepcomposer-samples/ar-cnn
# Create An Inference Object
inference_obj = Inference()
# Load The Checkpoint
checkpoint_var = '/content/aws-deepcomposer-samples/ar-cnn/best-model-epoch:0003.hdf5'
inference_obj.load_model(checkpoint_var) 

#### To choose a new input melody

1. In a different tab, switch back to the Jupyter console
2. Open the `sample_inputs` directory. It contains six sample melodies. 
3. Note the name of the file that you want to use, for example, 'new_world.midi'. 
4. In the following cell, replace **'sample_inputs/ode_to_joy.midi'** with the name of the file and run the cell below

In [ ]:
# Generate The Composition
input_melody = '/content/aws-deepcomposer-samples/ar-cnn/unconditional-fast.mid'  
inference_obj.generate_composition(input_melody, inference_params)

#### To listen to your composition

1. Run the following cell. It lists our compositions. 
2. Note the filename of the composition that you want to listen to, for example, "output_2.mid".

In [ ]:
!ls -ltr outputs/

3. In the following cell, replace **'outputs/output_0.mid'** with the name of the file and run the cell below

In [ ]:
output_melody = 'outputs/output_11.mid'
play_midi(output_melody)

>**NOTE**: Compositions are automatically saved.

## Evaluating your results

Now that you've generated a composition, let's find out how you did by running the code cells below. The code cells below provide you with some model metrics and a visualization of the piano rolls created.    

We'll analyze the composition using the following metrics: 

- *Empty bar rate:* The ratio of empty bars to the total number of bars. 
- *Pitch histogram distance:* The distribution and position of pitches.
- *In scale ratio:*  The ratio of the number of notes, in the C major key, to the total number of notes. 


### Visualizing the results
After computing the metrics, let's also visualize the *input piano roll* and compare it with the generated output piano roll to see which notes have been changed.

In [ ]:
# Input Midi Metrics:
print("The input midi metrics are:")
get_music_metrics(input_melody, beat_resolution=4)

print("\n")
# Generated Output Midi Metrics:
print("The generated output midi metrics are:")
get_music_metrics(output_melody, beat_resolution=4)

In [ ]:
# Convert The Input and Generated Midi To Tensors (a matrix)
input_pianoroll = process_midi(input_melody, beat_resolution=4)
output_pianoroll = process_midi(output_melody, beat_resolution=4)

In [ ]:
# Plot Input Piano Roll
plot_pianoroll(input_pianoroll, beat_resolution=4)

In [ ]:
# Plot Output Piano Roll
plot_pianoroll(output_pianoroll, beat_resolution=4)

## Submitting to the *Spin the Model* Chartbusters challnege

To submit your composition(s) and model to the *Spin the model* chartbusters challenge you will first need to create a public repository on [GitHub](https://github.com/). Then download your notebook, checkpoint files, and compositions from SageMaker, and upload them to your public repository. Use the link from your public repository to make your submission to the Chartbusters challenge! 